# SAR IMAGE CLASSIFICATION
- Classification problem
- Oceanography/Meteorology
- Pierre Tandeo (pierre.tandeo@imt-atlantique.fr), Ronan Fablet (ronan.fablet@imt-atlantique.fr), Carlos Granero (carlos.granero-belinchon@imt-atlantique.fr)

## Data and aim

In this project, we use SAR (Synthetic Aperture Radar) images (20km x 20km) of the ocean surface. The goal is to automatically classify different oceanic and atmospheric phenomena. Below, we will find an example of 10 classes corresponding to 10 different phenomena (among pure ocean swell in F, convective cells in I, icebergs in L, etc...):

![SAR](https://tandeo.files.wordpress.com/2019/01/sar_classes.png)

Here, we will use a database of 160 SAR images per class, anotated by experts. Data were collected in 2016 by the Sentinel-1 satellite.

## Evaluation and benchmark

We divided the database in two parts: "training" and "validation" folders with respectively 70% and 30% of the dataset. To learn the model (here a classifier), we propose to use cross-validations on the training dataset. Then, you will use the validation dataset to evaluate the performance of your model, using the total pourcentage of well predicted data. The reference result is **94% of accuracy** and is given by a deep leaning architecture. The reference paper is given below.

## Reference
- https://tandeo.files.wordpress.com/2019/11/wang_2019.pdf

### Import libraries

In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install imageio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import classical libraries
%matplotlib inline
%pylab inline

# figure size
plt.rcParams['figure.figsize'] = (16, 9)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['imread']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


### Prepare the datasets

Now, we load the training (70%) and validation (30%) datasets.

In [ ]:
!wget -O SAR.zip https://www.dropbox.com/s/c1vkkkxo33efpke/SAR.zip?dl=0

--2023-06-23 15:23:39--  https://www.dropbox.com/s/c1vkkkxo33efpke/SAR.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/c1vkkkxo33efpke/SAR.zip [following]
--2023-06-23 15:23:40--  https://www.dropbox.com/s/raw/c1vkkkxo33efpke/SAR.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd978711783d0beb572c667c096.dl.dropboxusercontent.com/cd/0/inline/B-gqr4Se5b8woOgdFZ7TPe7tlvTltCRdv4seHmTYaS8x39H1Qgp7f9XGh3rQEdVljpEModNo_ad2Yi80A4aYmQarKE0hZf8syRWV9PJdJLXsDU3rqUPFGmYLVEZcU4Yf-SEWl-rPhFawUjhXaCah-LRJ_02YklW2JKPxFDtDM_627Q/file# [following]
--2023-06-23 15:23:40--  https://ucd978711783d0beb572c667c096.dl.dropboxusercontent.com/cd/0/inline/B-gqr4Se5b8woOgdFZ7TPe7tlvTltCRdv4seHmTYaS8x39H1Qgp7f9XGh3rQEdVljpEModNo_ad2Yi80A4aYmQarKE

In [ ]:
!unzip SAR.zip
# We remove the zip file
rm SAR.zip?dl=0
rm SAR.zip

In [ ]:
# import functions
from os import listdir
from os.path import isfile, join
from imageio import imread
import sklearn

# paths to the training and validation datasets
path_train = "training/"
path_validation = "validation/"

# class names
classes = ['F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']

# initialization
X_train = []
y_train = []
X_validation = []
y_validation = []

# loop on images
for j in range(len(classes)):
    path = path_train + classes[j]
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # training
    for i in range(len(files)):
        tmp = imread(path + '/' + files[i])
        X_train.append(ravel(tmp[0:450,0:450:,0]))
        y_train.append(classes[j])

    # validation
    path = path_validation + classes[j]
    files = [f for f in listdir(path) if isfile(join(path, f))]
    for i in range(len(files)):
        tmp = imread(path + '/' + files[i])
        X_validation.append(ravel(tmp[0:450,0:450:,0]))
        y_validation.append(classes[j])

# transform to array
X_train = asarray(X_train)
y_train = asarray(y_train)
X_validation = asarray(X_validation)
y_validation = asarray(y_validation)

<ipython-input-35-046ea654e5b2>:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  tmp = imread(path + '/' + files[i])
<ipython-input-35-046ea654e5b2>:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  tmp = imread(path + '/' + files[i])


### Apply naive classifier

In [ ]:
# import functions
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score

# apply k-nearest classification
clf = NearestCentroid()
clf.fit(X_train, y_train)
y_predict = clf.predict(X_validation)

# compute average classifier score
print('Accuracy: '+ str(accuracy_score(y_validation, y_predict)))

# we are far from the 94% accuracy given a deep learning model!

Accuracy: 0.2625
